# 5. Consumer to Read Data and Business Logic to generate Offer

1. This program reads data from Kafka topic such as customer Id, latitude, longitude, timestamp.
2. Calculate distance of customer from a Mall.
3. If the distance between customer and mall is less than 1 km, fetch the customer data from Pizza_customers csv file.
4. Fetch the data from Offer csv file to determine the cluster, from which customer belongs to, based on income and spending score
5. Generate offer if customer belongs to one of the cluster, calculate offer for the customer based on cluster nomenclature

    a. Bronze :15% discount rate enabled them to be promoted to Silver tier

    b. Silver : We encourage them to give 5% discount as they are already under high transaction, and higher discount won’t affect the business much as their income is less

    c. Gold : 15% discount rate can be considered as an average rate in order to promote them to order more from our restaurant

    d.Diamond: We plan to give them our premium 25% discount rate

    d.Platinum : We plan to give them least discount rate of 5%
    
    
6. The generated output data saved in OfferOutput csv file

In [5]:
from kafka import KafkaConsumer
import json
from haversine import Unit
import haversine as hs
import pandas as pd
import csv


consumer =KafkaConsumer('CustData', group_id='None',bootstrap_servers=['localhost:9092'],auto_offset_reset='earliest')
header = ['ID' ,'Distance from Mall in Meter', 'Anual Income', 'Spending Score', 'Offer']

try:
    #Initialising OfferOutput csv file to store output
    with open('OfferOutput.csv', 'w', encoding='UTF8', newline='') as f:
        writer = csv.writer(f)

        # write the header
        writer.writerow(header)
        for msg in consumer:
            #Read data from Kafka topic
            #print('test1')
            x = msg.value.decode()
            #print('data:{0}\n'.format(i.value.decode()))
            final_dictionary = json.loads(x) 
            print(final_dictionary)
            final_dictionary = json.loads(x) 
            lati = final_dictionary["latitude"]
            longi = final_dictionary["longitude"]
                        
            CustOrdinates=(lati,longi) #Customer Coordinates
            MallOrdinates=(18.5187,73.9342) #Mall coordinates
            distance=hs.haversine(CustOrdinates,MallOrdinates,unit=Unit.METERS) #Calculate  distance between customer and Mall
           
            id = final_dictionary["custid"]
            #Read customer data based on ID
            data = pd.read_csv("pizza_customers.csv") 
            data1 = data[data.CustomerID == id]
            
            income= data1["Annual Income (k$)"].item()
            annual_spend= data1["Spending Score (1-100)"].item()
            offerText = ''
            #print("Income : ", income)
            #Check if cutomer data is available or not
            if(data1.dropna().empty):
                offerText="No Offer"
                print("empty")
            else:
                #check the distance, if distance lees than 1 km generate offer
                if(distance < 1000):
                    offer = pd.read_csv("Offer.csv")
                    #offer.reset_index(drop=True)
                    offerFilter = offer[(offer['Anual Income'] == income) & (offer['Spending Score'] == annual_spend)] 
                    cust_type = offerFilter["Offer"].item()
                    offerGiven=""
                    if(cust_type == "Silver"):
                        offerGiven = "5%" #Silver Customer
                    #print("Cluster 1 are a Silver customers contains " + str(datapoints) + " Datapoints")
                    elif(cust_type == "Bronze"):
                        offerGiven = "25%" #Bronze Customer

                    elif(cust_type == "Diamond"):    
                        offerGiven = "25%" #Diamond Customer

                    elif(cust_type == "Platinum"): 
                        offerGiven =  "5%"#Platinum Customer

                    elif(cust_type == "Gold"):
                        offerGiven = "15%" #Gold Customer
                    offerText = "Exlusive Offer for you! Get Flat "+ offerGiven +" off on your favourite pizza. Grab it soon!"  

                else:
                    offerText = "No Offer"
            #Save output in OfferOutput csv file
            data = [id, distance, income,annual_spend, offerText]
            writer.writerow(data)
            print("*********" + offerText + "*********" )
finally:
    print('close')
    consumer.close()

{'busline': '00001', 'key': '00001_f9408f65-0223-47b1-a742-61a90eb85a93', 'timestamp': '2022-02-28 16:35:04.866391', 'latitude': 18.5187, 'longitude': 73.9199, 'custid': 12, 'status': 'RUNNING'}
*********No Offer*********
{'busline': '00001', 'key': '00001_67641b22-1ab5-4306-9856-ee78510411f2', 'timestamp': '2022-02-28 16:54:58.269000', 'latitude': 18.5187, 'longitude': 73.9342, 'custid': 1, 'status': 'BROKEN'}
*********Exlusive Offer for you! Get Flat 15% off on your favourite pizza. Grab it soon!*********
{'busline': '00001', 'key': '00001_ae4e3d2e-e400-4537-99c0-e53145dbf12d', 'timestamp': '2022-02-28 16:55:01.286164', 'latitude': 18.5196, 'longitude': 73.9313, 'custid': 2, 'status': 'RUNNING'}
*********Exlusive Offer for you! Get Flat 5% off on your favourite pizza. Grab it soon!*********
{'busline': '00001', 'key': '00001_92eb0fad-480d-4809-8791-ae5b281abfc5', 'timestamp': '2022-02-28 16:55:04.299918', 'latitude': 18.5204, 'longitude': 73.9399, 'custid': 3, 'status': 'RUNNING'}
**

{'busline': '00001', 'key': '00001_17de4d65-d686-412f-908d-f97f411310ab', 'timestamp': '2022-02-28 16:56:31.562606', 'latitude': 18.5237, 'longitude': 73.9168, 'custid': 9, 'status': 'RUNNING'}
*********No Offer*********
{'busline': '00001', 'key': '00001_8ac5bb18-c15e-44f7-8567-f952503a7dde', 'timestamp': '2022-02-28 16:56:34.574235', 'latitude': 18.5244, 'longitude': 73.9179, 'custid': 10, 'status': 'RUNNING'}
*********No Offer*********
{'busline': '00001', 'key': '00001_2df4c54a-6275-4da1-b5e6-c873e3b1449a', 'timestamp': '2022-02-28 16:56:37.575890', 'latitude': 18.5245, 'longitude': 73.9189, 'custid': 11, 'status': 'BROKEN'}
*********No Offer*********
{'busline': '00001', 'key': '00001_b4e39702-9556-4d49-b21d-f466541893ca', 'timestamp': '2022-02-28 16:56:40.592164', 'latitude': 18.5187, 'longitude': 73.9199, 'custid': 12, 'status': 'RUNNING'}
*********No Offer*********
{'busline': '00001', 'key': '00001_8972c130-b088-4da8-bd22-f6704734e5ef', 'timestamp': '2022-02-28 16:56:43.608203

KeyboardInterrupt: 